In [1]:
import pandas as pd
import requests as req
import numpy as np
import re

In [2]:
# objectives: get longitude, latitude, zipcode

In [3]:
res = req.get('https://nominatim.openstreetmap.org/search?street=181 Front Street&state=New York&city=Brooklyn&addressdetails=1&format=json')
res.json()

[{'place_id': 238740394,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 684690680,
  'boundingbox': ['40.7024618', '40.7026248', '-73.9863777', '-73.9855975'],
  'lat': '40.70252645',
  'lon': '-73.98598783528503',
  'display_name': '181 Front Street, Front Street, Brooklyn, Kings County, New York, 11201, United States',
  'class': 'building',
  'type': 'yes',
  'importance': 0.611,
  'address': {'building': '181 Front Street',
   'road': 'Front Street',
   'suburb': 'Brooklyn',
   'city': 'New York',
   'state': 'New York',
   'ISO3166-2-lvl4': 'US-NY',
   'postcode': '11201',
   'country': 'United States',
   'country_code': 'us'}}]

In [4]:
fakeaddy = '48C Winham Avenue'
fakeaddy = re.sub('([a-zA-Z0-9]{1,}-)*FLOOR', '', str(fakeaddy).strip(), flags=re.IGNORECASE).strip()
fakeaddy = re.sub('\s{2,}', ' ', fakeaddy.strip(), flags=re.IGNORECASE).strip()
fakeaddy

'48C Winham Avenue'

In [5]:
fake_zip = '1000:5000'
re.sub(':.*','',fake_zip, flags=re.IGNORECASE).strip()

'1000'

In [6]:
def remove_floor_info(addy):
    new_addy = re.sub('([a-zA-Z0-9]{1,}-)*FLOOR', '', str(addy).strip(), flags=re.IGNORECASE).strip()
    new_addy = re.sub('\s{2,}', ' ', new_addy.strip(), flags=re.IGNORECASE).strip()
    return new_addy

In [15]:
def get_loc_data(street, boro):
    state = "New York"
    city = str(boro).strip()
    address = remove_floor_info(str(street).strip())
    query = f"https://nominatim.openstreetmap.org/search?street={address}&state={state}&city={city}&addressdetails=1&limit=1&format=json"
    result = req.get(query).json()
    
    if len(result) < 1:
        return [np.NaN, np.NaN, np.NaN, address]

    data = result[0]
    zipcode = None
    if 'postcode' in data['address'].keys():
        zipcode = re.sub(':.*','', data['address']['postcode'].strip(), flags=re.IGNORECASE).strip()
    elif data['display_name']:
        zipcode = data['display_name'].split(',')[-2].strip()
        zipcode = re.sub(':.*','', zipcode, flags=re.IGNORECASE).strip()

    res = [float(data['lon'].strip()), float(data['lat'].strip()), zipcode, address]
    return res

In [8]:
stat_csv = '../Preprocessing/ImputedSets/staten_imputed.csv'
bx_csv = '../Preprocessing/ImputedSets/bronx_imputed.csv'
manhatt_csv = '../Preprocessing/ImputedSets/manhat_imputed.csv'
queens_csv = '../Preprocessing/ImputedSets/queens_imputed.csv'
bklyn_csv = '../Preprocessing/ImputedSets/brook_imputed.csv'

source_csvs = [stat_csv, bx_csv, manhatt_csv, queens_csv, bklyn_csv]
dest_paths = ['./DFwLoc/stat_df.csv', './DFwLoc/bronx_df.csv', './DFwLoc/manhatt_df.csv', './DFwLoc/queens_df.csv', './DFwLoc/bklyn_df.csv']
boros = ['Staten-Island', 'Bronx', 'Manhattan', 'Queens', 'Brooklyn']

In [17]:
def get_loc_for_all(df_source, df_dest, boro):
    print(f"getting {boro}...")
    daf = pd.DataFrame(pd.read_csv(df_source))
    daf[['Long', 'Lat', 'ZipCode']] = None

    for idx, row in daf.iterrows():
        data = get_loc_data(row['Address'], boro)
        daf.at[idx, 'Address'] = data[-1]
        daf.at[idx, 'ZipCode'] = data[2]
        daf.at[idx, 'Lat'] = data[1]
        daf.at[idx, 'Long'] = data[0]
    
    daf.to_csv(df_dest, index=False)
    return

In [18]:
for i in range(2, len(source_csvs), 1):
    get_loc_for_all(df_source=source_csvs[i], df_dest=dest_paths[i], boro=boros[i])
    

getting Manhattan...
getting Queens...
getting Brooklyn...
